In [1]:
import numpy as np
import dedalus.public as d3

In [2]:
# Domain
coords = d3.CartesianCoordinates('x', 'y', 'z')
dist = d3.Distributor(coords, dtype=np.float64)
xbasis = d3.RealFourier(coords['x'], 64, bounds=(-1, 1))
ybasis = d3.RealFourier(coords['y'], 64, bounds=(-1, 1))
zbasis = d3.RealFourier(coords['z'], 64, bounds=(-1, 1))

In [3]:
# Fields
f1 = dist.Field(name='f1', bases=(xbasis, ybasis, zbasis))
f2 = dist.Field(name='f2', bases=(xbasis, ybasis, zbasis))
df1 = dist.Field(name='df1', bases=(xbasis, ybasis, zbasis))
df2 = dist.Field(name='df2', bases=(xbasis, ybasis, zbasis))
dg = dist.Field(name='dg', bases=(xbasis, ybasis, zbasis))
dg.adjoint = True

In [4]:
# Nested operator with large expression swell
g1 = f1 * f2
g2 = g1 * g1
g3 = g2 * g2
G = g3 * g3

g1.name = 'g1'
g2.name = 'g2'
g3.name = 'g3'
G.name = 'G'

# Current symbolic Frechet derivative, with expression swell
dG = G.frechet_differential([f1, f2], [df1, df2])
print("Symbolic derivative:", dG)

# Test JVP vs symbolic Frechet derivative on random inputs
f1.fill_random('g'); f1['c']; f1['g']
f2.fill_random('g'); f2['c']; f1['g']
df1.fill_random('g'); df1['c']; df1['g']
df2.fill_random('g'); df2['c']; df2['g']

g, dg_fwd = G.evaluate_jvp({f1: df1, f2:df2})
dg_fwd = dg_fwd.copy()
dg_sym = dG.evaluate()
dg_sym = dg_sym.copy()

print("JVP matches symbolic:", np.allclose(dg_fwd['g'], dg_sym['g']))

Symbolic derivative: (((df1*f2 + f1*df2)*f1*f2 + f1*f2*(df1*f2 + f1*df2))*f1*f2*f1*f2 + f1*f2*f1*f2*((df1*f2 + f1*df2)*f1*f2 + f1*f2*(df1*f2 + f1*df2)))*f1*f2*f1*f2*f1*f2*f1*f2 + f1*f2*f1*f2*f1*f2*f1*f2*(((df1*f2 + f1*df2)*f1*f2 + f1*f2*(df1*f2 + f1*df2))*f1*f2*f1*f2 + f1*f2*f1*f2*((df1*f2 + f1*df2)*f1*f2 + f1*f2*(df1*f2 + f1*df2)))
JVP matches symbolic: True


In [8]:
# Test VJP on random inputs
def inner(a, b):
    return np.sum(a['g'] * b['g'])
dg.fill_random('g'); dg['c']; dg['g']

g, df_rev = G.evaluate_vjp({G: dg}, id=np.random.randint(0, 1000000))

print("Inner products <random | Jacobian | random>:")
print("<R2 | J @ R1>  :", inner(dg, dg_fwd))
print("<J.T @ R2 | R1>:", inner(df1, df_rev[f1]) + inner(df2, df_rev[f2]))

Inner products <random | Jacobian | random>:
<R2 | J @ R1>  : -309017309.33200324
<J.T @ R2 | R1>: -309017309.332003


In [9]:
df_rev

{<Field 5200552512>: <Field 5200249584>,
 <Field 5200159440>: <Field 5193695792>}

In [8]:
print("Time forward evaluation:")
%timeit G.evaluate(id=np.random.randint(0, 1000000))

Time forward evaluation:
469 µs ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
print("Time JVP evaluation:")
%timeit G.evaluate_jvp({f1: df1, f2:df2}, id=np.random.randint(0, 1000000))

Time JVP evaluation:
1.61 ms ± 66.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
print("Time symbolic derivative evaluation:")
%timeit dG.evaluate(id=np.random.randint(0, 1000000))

Time symbolic derivative evaluation:
11.2 ms ± 542 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print("Time VJP evaluation:")
%timeit g, df_rev = G.evaluate_vjp(dg, id=np.random.randint(0, 1000000))

Time VJP evaluation:
2.67 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
